## Census EDA

#### Data Collection:
To get the data, copy example code from this site: https://api.census.gov/data/2017/acs/acs5/profile/examples.html

Variables are at this site:
https://api.census.gov/data/2017/acs/acs5/profile/variables.html

Include your API_key and you should be good to go!

In [1]:
import numpy as np
import requests
import json
from us import states
import pandas as pd
import os
from shapely.geometry import box
import geopandas as gpd
import earthpy as et
from earthpy import clip as cl



In [2]:
API_KEY = open('api_key','r')
API_KEY=API_KEY.read()
API_KEY = API_KEY.strip()

In [3]:
states.NY.fips, states.OH.fips

('36', '39')

In [4]:
ohio_url = "https://api.census.gov/data/2017/acs/acs5/profile?get=DP03_0062E,DP05_0033E,DP05_0064E,DP05_0064PE,DP05_0065PE,DP02_0015E,DP04_0024PE,DP04_0023PE,NAME&for=tract:*&in=state:39&key="
url_fin = ohio_url+API_KEY
ohio_response = requests.get(ohio_url)

ny_url = "https://api.census.gov/data/2017/acs/acs5/profile?get=DP03_0062E,DP05_0033E,DP05_0064E,DP05_0064PE,DP05_0065PE,DP02_0015E,DP04_0024PE,DP04_0023PE,NAME&for=tract:*&in=state:36&key="
url_fin = ny_url+API_KEY
ny_response = requests.get(ny_url)



ny_response


<Response [200]>

In [5]:
df_ohio = pd.DataFrame(ohio_response.json())
df_ohio.columns = ['Median_hh_income','Total_population',
              'Total_white_population','Percent_white_population',
              'Percent_black_population','Average_hh_size',
              'Percent_hh_built_1950s','Percent_hh_built_1960s',
              'NAME','state','county','tract']
df_ohio = df_ohio[1:]



In [6]:
df_ny = pd.DataFrame(ny_response.json())
df_ny.columns = ['Median_hh_income','Total_population',
              'Total_white_population','Percent_white_population',
              'Percent_black_population','Average_hh_size',
              'Percent_hh_built_1950s','Percent_hh_built_1960s',
              'NAME','state','county','tract']
df_ny = df_ny[1:]

In [7]:
df_ny.head()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,NAME,state,county,tract
1,64311,6726,3454,51.4,11.5,2.77,27.1,27.9,"Census Tract 134, Westchester County, New York",36,119,013400
2,84063,3889,2534,65.2,5.6,3.25,21.0,6.2,"Census Tract 135, Westchester County, New York",36,119,013500
3,63418,3248,2332,71.8,8.7,1.88,27.2,27.7,"Census Tract 146.05, Westchester County, New York",36,119,014605
4,-666666666,21,21,100.0,0.0,-666666666.00,0.0,0.0,"Census Tract 1.04, Westchester County, New York",36,119,000104
5,168500,2920,2210,75.7,12.5,3.04,64.0,11.1,"Census Tract 111.02, Westchester County, New York",36,119,011102


In [8]:
len(df_ny), len(df_ohio)

(4918, 2952)

In [9]:
for y in [df_ohio, df_ny]:
    for x in y.columns:
        if x == "NAME":
            break

        y[x]=y[x].astype(float)

In [10]:
df_ohio['suburb_house_decades']=df_ohio['Percent_hh_built_1950s']+df_ohio['Percent_hh_built_1960s']
df_ny['suburb_house_decades']=df_ny['Percent_hh_built_1950s']+df_ny['Percent_hh_built_1960s']



In [11]:
df_ny.describe()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,suburb_house_decades
count,4.918000e+03,4918.000000,4918.000000,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03
mean,-1.647080e+07,4025.666531,2662.047580,-8.404436e+06,-8.404482e+06,-1.260675e+07,-1.111561e+07,-1.111562e+07,-2.223123e+07
std,1.037119e+08,1978.053236,1841.131288,7.438744e+07,7.438743e+07,9.081436e+07,8.537180e+07,8.537180e+07,1.707436e+08
min,-6.666667e+08,0.000000,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-1.333333e+09
25%,4.439875e+04,2638.750000,1183.250000,3.830000e+01,1.800000e+00,2.330000e+00,6.100000e+00,5.300000e+00,1.360000e+01
50%,6.112900e+04,3800.000000,2435.000000,7.740000e+01,5.700000e+00,2.620000e+00,1.110000e+01,9.100000e+00,2.220000e+01
75%,8.434375e+04,5120.750000,3857.750000,9.310000e+01,2.380000e+01,3.030000e+00,2.010000e+01,1.560000e+01,3.780000e+01
max,2.500010e+05,28937.000000,14748.000000,1.000000e+02,1.000000e+02,6.540000e+00,1.000000e+02,1.000000e+02,1.000000e+02


In [12]:
df_ohio.describe()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,suburb_house_decades
count,2.952000e+03,2952.000000,2952.000000,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03
mean,-3.786512e+06,3932.844173,3315.944106,-2.709947e+06,-2.710009e+06,-3.161696e+06,-2.935848e+06,-2.935850e+06,-5.871698e+06
std,5.045789e+07,1898.519945,1963.276763,4.242583e+07,4.242582e+07,4.580953e+07,4.415068e+07,4.415068e+07,8.830137e+07
min,-6.666667e+08,0.000000,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-1.333333e+09
25%,3.584575e+04,2608.500000,1927.500000,7.360000e+01,1.600000e+00,2.240000e+00,6.500000e+00,5.875000e+00,1.430000e+01
50%,5.026400e+04,3719.000000,3231.500000,9.210000e+01,5.500000e+00,2.440000e+00,1.160000e+01,9.800000e+00,2.270000e+01
75%,6.423375e+04,4949.500000,4474.500000,9.770000e+01,2.222500e+01,2.620000e+00,1.980000e+01,1.582500e+01,3.632500e+01
max,2.221880e+05,20372.000000,19409.000000,1.000000e+02,1.000000e+02,4.640000e+00,8.060000e+01,9.020000e+01,9.370000e+01


In [13]:
df_ohio['Class']=np.where(
    ((df_ohio['Median_hh_income']> np.percentile(df_ohio['Median_hh_income'],50)) &
    (df_ohio['Percent_white_population']> np.percentile(df_ohio['Percent_white_population'],50)) &
    (df_ohio['Percent_white_population']> np.percentile(df_ohio['Percent_white_population'],50))) 
    ,1,0)


In [14]:
for x in [df_ohio, df_ny]:
    x['Typology_Flag']=np.where(
        ((x['Median_hh_income']> np.percentile(x['Median_hh_income'],50)) &
        (x['Percent_white_population']> np.percentile(x['Percent_white_population'],50)) &
        (x['Percent_white_population']> np.percentile(x['Percent_white_population'],50))) 
        ,1,0)

    x['Housing_Age_Flag']=np.where(
        (x['suburb_house_decades']> np.percentile(x['suburb_house_decades'],50)) 
        ,1,0)
    
    x['Combined_Flag']=np.where(
        ((x['Typology_Flag']==1) &
        (x['Housing_Age_Flag']==1)) 
        ,1,0)




In [15]:
df_ohio.describe()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,suburb_house_decades,Class,Typology_Flag,Housing_Age_Flag,Combined_Flag
count,2.952000e+03,2952.000000,2952.000000,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03,2.952000e+03,2952.000000,2952.000000,2952.000000,2952.000000
mean,-3.786512e+06,3932.844173,3315.944106,-2.709947e+06,-2.710009e+06,-3.161696e+06,-2.935848e+06,-2.935850e+06,-5.871698e+06,0.333333,0.333333,0.499322,0.154133
std,5.045789e+07,1898.519945,1963.276763,4.242583e+07,4.242582e+07,4.580953e+07,4.415068e+07,4.415068e+07,8.830137e+07,0.471484,0.471484,0.500084,0.361137
min,-6.666667e+08,0.000000,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-1.333333e+09,0.000000,0.000000,0.000000,0.000000
25%,3.584575e+04,2608.500000,1927.500000,7.360000e+01,1.600000e+00,2.240000e+00,6.500000e+00,5.875000e+00,1.430000e+01,0.000000,0.000000,0.000000,0.000000
50%,5.026400e+04,3719.000000,3231.500000,9.210000e+01,5.500000e+00,2.440000e+00,1.160000e+01,9.800000e+00,2.270000e+01,0.000000,0.000000,0.000000,0.000000
75%,6.423375e+04,4949.500000,4474.500000,9.770000e+01,2.222500e+01,2.620000e+00,1.980000e+01,1.582500e+01,3.632500e+01,1.000000,1.000000,1.000000,0.000000
max,2.221880e+05,20372.000000,19409.000000,1.000000e+02,1.000000e+02,4.640000e+00,8.060000e+01,9.020000e+01,9.370000e+01,1.000000,1.000000,1.000000,1.000000


In [16]:
df_ny.describe()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,suburb_house_decades,Typology_Flag,Housing_Age_Flag,Combined_Flag
count,4.918000e+03,4918.000000,4918.000000,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03,4.918000e+03,4918.000000,4918.00000,4918.000000
mean,-1.647080e+07,4025.666531,2662.047580,-8.404436e+06,-8.404482e+06,-1.260675e+07,-1.111561e+07,-1.111562e+07,-2.223123e+07,0.303579,0.49939,0.191948
std,1.037119e+08,1978.053236,1841.131288,7.438744e+07,7.438743e+07,9.081436e+07,8.537180e+07,8.537180e+07,1.707436e+08,0.459850,0.50005,0.393872
min,-6.666667e+08,0.000000,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-1.333333e+09,0.000000,0.00000,0.000000
25%,4.439875e+04,2638.750000,1183.250000,3.830000e+01,1.800000e+00,2.330000e+00,6.100000e+00,5.300000e+00,1.360000e+01,0.000000,0.00000,0.000000
50%,6.112900e+04,3800.000000,2435.000000,7.740000e+01,5.700000e+00,2.620000e+00,1.110000e+01,9.100000e+00,2.220000e+01,0.000000,0.00000,0.000000
75%,8.434375e+04,5120.750000,3857.750000,9.310000e+01,2.380000e+01,3.030000e+00,2.010000e+01,1.560000e+01,3.780000e+01,1.000000,1.00000,0.000000
max,2.500010e+05,28937.000000,14748.000000,1.000000e+02,1.000000e+02,6.540000e+00,1.000000e+02,1.000000e+02,1.000000e+02,1.000000,1.00000,1.000000


In [21]:
df_ohio.columns

Index(['Median_hh_income', 'Total_population', 'Total_white_population',
       'Percent_white_population', 'Percent_black_population',
       'Average_hh_size', 'Percent_hh_built_1950s', 'Percent_hh_built_1960s',
       'NAME', 'state', 'county', 'tract', 'suburb_house_decades', 'Class',
       'Typology_Flag', 'Housing_Age_Flag', 'Combined_Flag'],
      dtype='object')

In [24]:
df_ohio['GEOID']=df_ohio['state']+df_ohio['county']+df_ohio['tract']
df_ny['GEOID']=df_ny['state']+df_ny['county']+df_ny['tract']

In [27]:
df_ohio.head()

,Median_hh_income,Total_population,Total_white_population,Percent_white_population,Percent_black_population,Average_hh_size,Percent_hh_built_1950s,Percent_hh_built_1960s,NAME,state,county,tract,suburb_house_decades,Class,Typology_Flag,Housing_Age_Flag,Combined_Flag,GEOID
1,52061.0,2657.0,2426.0,91.3,5.8,1.83,5.5,3.5,"Census Tract 5201.06, Summit County, Ohio",39,153,520106,9.0,0,0,0,0,39153520106
2,61696.0,2171.0,2000.0,92.1,8.4,2.50,19.7,12.7,"Census Tract 8109, Mahoning County, Ohio",39,099,810900,32.4,0,0,1,0,39099810900
3,27686.0,4461.0,4256.0,95.4,5.1,2.35,3.9,3.8,"Census Tract 9728, Athens County, Ohio",39,009,972800,7.7,0,0,0,0,39009972800
4,15720.0,4504.0,4200.0,93.3,5.4,2.47,4.3,6.7,"Census Tract 9731.01, Athens County, Ohio",39,009,973101,11.0,0,0,0,0,39009973101
5,45795.0,4026.0,3859.0,95.9,3.8,2.53,3.7,8.8,"Census Tract 9737, Athens County, Ohio",39,009,973700,12.5,0,0,0,0,39009973700


In [25]:
df_ohio.to_csv("Ohio_census_data.csv")

In [26]:
df_ny.to_csv("New_York_census_data.csv")

In [61]:
ohio_bound = gpd.read_file('./../ohio_census_tract/cb_2017_39_tract_500k.shp')
ny_bound = gpd.read_file('./../newyork_census_tract/cb_2017_36_tract_500k.shp')




DriverError: ./../newyork_census_tract/cb_2017_36_tract_500k.shp: No such file or directory

In [53]:
len(ohio_bound), len(df_ohio)

(2948, 2952)

In [54]:
len(ny_bound), len(df_ny)

(4906, 4918)

In [55]:
ohio_bound_join = pd.merge(ohio_bound, df_ohio,on="GEOID")
ny_bound_join = pd.merge(ny_bound, df_ny,on="GEOID")


In [56]:
len(ohio_bound_join), len(ny_bound_join)

(2948, 4906)

In [58]:
ohio_bound_join.to_file('./../ohio_census_tract/cb_2017_39_tract_500k.shp')
ny_bound_join.to_file('./../newyork_census_tract/cb_2017_39_tract_500k.shp')


